In [ ]:
import torch
import random
import calendar

import platform
import sys

# Проверка доступности GPU (как мы делали раньше)
# Если CUDA есть, используем её. Если нет - процессор.

print(f"torch.version.cuda: {torch.version.cuda}")       # Должно показать '12.4' или '12.1' (не None!)
print(f"torch.cuda.is_available(): {torch.cuda.is_available()}") # Должно быть True
print(f"torch.xpu.is_available(): {torch.xpu.is_available()}") 

def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    # Проверяем атрибут xpu, чтобы код не падал на машинах со старым PyTorch
    elif hasattr(torch, 'xpu') and torch.xpu.is_available():
        return torch.device("xpu")
    else:
        return torch.device("cpu")

device = get_device()


print(f"✅ Устройство для вычислений: {device}")
# Если XPU активен, полезно вывести имя карты:
if device.type == 'xpu':
    print(f"   Карта: {torch.xpu.get_device_name(0)}")
elif device.type == 'cuda':
    print(f"   Карта: {torch.cuda.get_device_name(0)}")


In [ ]:
print(f"sys.version: {sys.version}")

print(f"platform.python_version(): {platform.python_version()}")

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
          'Sep', 'Oct', 'Nov', 'Dec']


def generate_date():
    year = random.randint(1000, 2100)

    monthIdx = random.randint(1, 12)

    match monthIdx:
        case 1 | 3 | 5 | 7 | 8 | 10 | 12:
            day = random.randint(1, 31)
        case 4 | 6 | 9 | 11:
            day = random.randint(1, 30)
        case 2:
            if calendar.isleap(year):
                day = random.randint(1, 29)
            else:
                day = random.randint(1, 28)

    # 1. Порядок: Год - Месяц - День
    # 2. Форматирование :02d добавляет ноль в начале, если число < 10 (например, 05)
    tgt_str = f'{year}-{monthIdx:02d}-{day:02d}'

    # Вход оставляем как есть: "25 Jan 2023"
    src_str = f'{day} {months[monthIdx-1]} {year}'

    # Добавляем маркеры старта и стопа
    tgt_str = f'<{tgt_str}>'

    return src_str, tgt_str


print(generate_date())

In [ ]:
TOTAL_SAMPLES = 50000
TRAIN_RATIO = 0.8

print(f"🔄 Генерируем {TOTAL_SAMPLES} уникальных примеров (годы 1000-2100)...")

dataset_set = set()

# Теперь этот цикл не зависнет, так как пространство вариантов (400k) > цели (50k)
while len(dataset_set) < TOTAL_SAMPLES:
    dataset_set.add(generate_date())

full_dataset = list(dataset_set)
random.shuffle(full_dataset)

split_index = int(TOTAL_SAMPLES * TRAIN_RATIO)
train_data = full_dataset[:split_index]
test_data = full_dataset[split_index:]

print(f"✅ Готово! Train: {len(train_data)}, Test: {len(test_data)}")


In [ ]:
# 0. Определяем специальные токены и их индексы (зафиксируем их для удобства)
# Важно: 0-й индекс часто резервируют для PAD, так как он используется по умолчанию
# при инициализации тензоров нулями.
PAD_token = 0
SOS_token = 1 
EOS_token = 2

# Символы, которые гарантированно будут в нашем словаре
special_chars = {
    '<PAD>': PAD_token, 
    '<SOS>': SOS_token, 
    '<EOS>': EOS_token
}


# 1. Определяем все возможные символы (Alphabet)
# set() убирает дубликаты
chars = set("0123456789 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ<>-")

# 2. Объединяем символы и специальные токены для создания словаря char2idx
# Начинаем индексацию с 3 (после 0, 1, 2)
sorted_data_chars = sorted(chars)
next_index = len(special_chars)

# 3. Создаем Map: Символ -> Индекс (Integer)
# sorted() сортирует, чтобы порядок всегда был одинаковым
# enumerate() возвращает пары (счетчик, элемент)
char2idx = special_chars.copy()
char2idx.update({char: idx + next_index for idx, char in enumerate(sorted_data_chars)})
print(f"char2idx: {char2idx}")

# 3. Создаем обратный Map: Индекс -> Символ (для расшифровки ответа)
idx2char = {idx: char for char, idx in char2idx.items()}

print(f"idx2char: {idx2char}")


# Добавляем служебный токен "EOS" (End Of Sentence) или PAD, если нужно
# Пока просто запомним размер нашего "алфавита"
vocab_size = len(char2idx)

print(f"Размер словаря (количество уникальных символов): {vocab_size}")
print(f"Индекс буквы 'A': {char2idx['A']}")
print(f"Символ под индексом 10: {idx2char[10]}")
print(f"Размер словаря с токенами: {vocab_size}")
print(f"Индекс <PAD>: {char2idx['<PAD>']}")
print(f"Индекс <EOS>: {char2idx['<EOS>']}")

In [ ]:
import torch.nn as nn


from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class EncoderRNN_Packed(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN_Packed, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    # Добавили аргумент lengths (длины фраз в батче)
    def forward(self, input, lengths, hidden):
        embedded = self.embedding(input)
        
        # 1. УПАКОВКА (PACKING)
        # Мы говорим RNN: "Игнорируй хвосты!"
        # enforce_sorted=False позволяет не сортировать батч по длине (удобно!)
        packed = pack_padded_sequence(embedded, lengths.cpu(), enforce_sorted=False)
        
        # 2. ПРОГОН (уже оптимизированный)
        output, hidden = self.gru(packed, hidden)
        
        # 3. РАСПАКОВКА (UNPACKING)
        # Возвращаем всё в привычный вид (с нулями), чтобы Декодер мог работать
        output, _ = pad_packed_sequence(output)
        
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

print("✅ Packed Encoder готов.")

In [ ]:
import torch.nn.functional as F

class AttnDecoderRNN_Masked(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=15, dropout_p=0.1):
        super(AttnDecoderRNN_Masked, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    # Добавили аргумент encoder_mask
    def forward(self, input, hidden, encoder_outputs, encoder_mask):
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)

        attn_input = torch.cat((embedded[0], hidden[0]), 1) 
        
        # 1. Считаем "сырые" энергии (до Softmax)
        attn_energies = self.attn(attn_input) # (Batch, Max_Len)

        # 2. ПРИМЕНЯЕМ МАСКУ!
        if encoder_mask is not None:
            # Там, где маска == 0 (PAD), ставим -1 миллиард.
            # Softmax(-1e9) -> 0.0
            attn_energies = attn_energies.masked_fill(encoder_mask == 0, -1e9)

        # 3. Теперь безопасно делаем Softmax
        attn_weights = F.softmax(attn_energies, dim=1)

        encoder_outputs_transposed = encoder_outputs.transpose(0, 1)
        
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),
                                 encoder_outputs_transposed)

        output = torch.cat((embedded[0], attn_applied[:, 0, :]), 1)
        output = self.attn_combine(output).unsqueeze(0) 
        output = F.relu(output)
        
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        
        return output, hidden, attn_weights

print("✅ Masked Decoder готов.")

In [ ]:
# Маркер конца строки (End Of Sentence).
# В нашем генераторе мы оборачиваем дату в <...>.
# Значит, символом остановки будет '>'.
EOS_token = char2idx['>']

def tensorFromSentence(sentence):
    # 1. Превращаем строку в список индексов
    indexes = [char2idx[char] for char in sentence]
    
    # 2. Добавляем маркер конца (опционально, но полезно для декодера)
    # В нашем случае '>' уже есть в конце строки от генератора, 
    # но для надежности можно убедиться, что он там есть.
    # Наш генератор выдает '<...>', так что '>' уже внутри indexes.
    
    # 3. Превращаем в Тензор
    # dtype=torch.long обязателен для индексов (Embedding слой требует long)
    # .view(-1, 1) делает из вектора столбик (Sequence Length x Batch Size)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [ ]:
def train_batch_packed(input_batch, target_batch, encoder, decoder, 
                       encoder_optimizer, decoder_optimizer, criterion):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    batch_size = input_batch.size(1)
    target_len = target_batch.size(0)

    # --- СЧИТАЕМ ДЛИНЫ ---
    # input_batch: (Seq_Len, Batch). Транспонируем -> (Batch, Seq_Len)
    # Считаем, сколько НЕ-нулей в каждой строке
    # .cpu() обязательно для pack_padded_sequence в некоторых версиях
    lengths = (input_batch.transpose(0, 1) != PAD_token).sum(dim=1).cpu()
    
    # ---------------------

    encoder_hidden = encoder.initHidden(batch_size)
    
    # ПЕРЕДАЕМ ДЛИНЫ В ЭНКОДЕР!
    encoder_outputs, encoder_hidden = encoder(input_batch, lengths, encoder_hidden)
    
    # Маска для Attention (та же логика, что и раньше)
    mask = (input_batch.transpose(0, 1) != PAD_token)
    full_mask = torch.zeros(batch_size, decoder.max_length, dtype=torch.bool, device=device)
    actual_len = mask.size(1)
    limit_len = min(actual_len, decoder.max_length)
    full_mask[:, :limit_len] = mask[:, :limit_len]
    
    # Pad encoder outputs
    proj_encoder_outputs = torch.zeros(decoder.max_length, batch_size, encoder.hidden_size, device=device)
    # output от pad_packed_sequence может быть короче max_length, если все фразы короткие
    out_len = encoder_outputs.size(0) 
    limit_copy = min(out_len, decoder.max_length)
    proj_encoder_outputs[:limit_copy, :, :] = encoder_outputs[:limit_copy, :, :]

    # Декодер (без изменений)
    decoder_input = torch.tensor([[SOS_token] * batch_size], device=device)
    decoder_hidden = encoder_hidden
    loss = torch.tensor(0.0, device=device)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_len):
            decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, proj_encoder_outputs, encoder_mask=full_mask)
            loss += criterion(decoder_output, target_batch[di])
            decoder_input = target_batch[di].unsqueeze(0) 
    else:
        for di in range(target_len):
            decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, proj_encoder_outputs, encoder_mask=full_mask)
            loss += criterion(decoder_output, target_batch[di])
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.transpose(0, 1).detach()

    loss.backward()
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), 5.0)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(), 5.0)
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_len

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# Вспомогательная функция: Строка -> Список индексов
def indexesFromSentence(sentence):
    # Удаляем старые маркеры < и >, если они есть, чтобы не путаться
    clean_sentence = sentence.replace('<', '').replace('>', '')
    
    # Превращаем в цифры
    # Если вдруг встретим незнакомый символ, пропускаем его (или падаем, но пока пропустим)
    return [char2idx[char] for char in clean_sentence if char in char2idx] + [EOS_token]

def get_batch(batch_size=32):
    input_list = []
    target_list = []
    
    # 1. Набираем batch_size случайных примеров
    for _ in range(batch_size):
        pair = random.choice(train_data) # Берем из обучающей выборки
        input_str, target_str = pair
        
        # Превращаем в списки чисел (НЕ тензоры пока)
        input_idxs = indexesFromSentence(input_str)
        target_idxs = indexesFromSentence(target_str)
        
        # Важно: для PyTorch конвертируем списки в тензоры
        input_list.append(torch.tensor(input_idxs, dtype=torch.long, device=device))
        target_list.append(torch.tensor(target_idxs, dtype=torch.long, device=device))
        
    # 2. Магия PAD_SEQUENCE
    # Она берет список тензоров разной длины и делает из них один прямоугольный тензор.
    # padding_value=PAD_token (наш 0) — чем заполнять пустоту.
    # ВАЖНО: По умолчанию она делает размерность (Max_Length, Batch_Size).
    # Это то, что нужно для RNN (время идет вниз, батчи идут вправо).
    input_batch = pad_sequence(input_list, padding_value=PAD_token)
    target_batch = pad_sequence(target_list, padding_value=PAD_token)
    
    return input_batch, target_batch

# === ТЕСТ ===
# Давай посмотрим, как выглядит один батч
inp, tgt = get_batch(batch_size=3)
print(f"Форма входного батча (Seq_Len, Batch): {inp.shape}")
print(f"Форма целевого батча (Seq_Len, Batch): {tgt.shape}")
print("\nПример входного тензора (видим нули-паддинги в конце коротких фраз?):")
print(inp.transpose(0, 1)) # Транспонируем для удобного просмотра (Batch, Seq)

In [ ]:
# === ЭТАП ПРЕД-ЗАГРУЗКИ (ETL) ===
# Мы подготовим данные один раз и навсегда, чтобы не тратить CPU в цикле.

print("🔄 Подготовка данных в VRAM...")

def prepare_full_dataset_tensor(data_list):
    input_tensors = []
    target_tensors = []
    
    for src, tgt in data_list:
        input_tensors.append(torch.tensor(indexesFromSentence(src), dtype=torch.long))
        target_tensors.append(torch.tensor(indexesFromSentence(tgt), dtype=torch.long))
    
    # Делаем один гигантский Pad для всего датасета
    # (Total_Samples, Max_Len)
    # ВАЖНО: pad_sequence по умолчанию делает (Len, Batch).
    # Нам удобнее (Batch, Len) для хранения, а в цикле транспонируем.
    input_full = pad_sequence(input_tensors, batch_first=True, padding_value=PAD_token)
    target_full = pad_sequence(target_tensors, batch_first=True, padding_value=PAD_token)
    
    return input_full.to(device), target_full.to(device)

# 1. Загружаем ВСЕ данные сразу на видеокарту
train_input_gpu, train_target_gpu = prepare_full_dataset_tensor(train_data)

print(f"✅ Данные загружены в VRAM GPU!")
print(f"Размер входа: {train_input_gpu.shape} (Примеров, Длина)")
print(f"Занимает памяти: {train_input_gpu.element_size() * train_input_gpu.numel() / 1024 / 1024:.2f} MB")

In [ ]:
import torch.optim as optim
import time

# === CONFIG ===
BATCH_SIZE = 64
n_iters = 30000  # Попробуем 10k шагов. С маской должно хватить.
learning_rate = 0.001
teacher_forcing_ratio = 0.5

# Инициализация (Новый декодер!)
encoder_batch = EncoderRNN_Packed(vocab_size, 128).to(device)
decoder_batch = AttnDecoderRNN_Masked(128, vocab_size, max_length=15, dropout_p=0.1).to(device)

torch.set_float32_matmul_precision('high')
encoder_batch.compile();
decoder_batch.compile();

enc_optim = optim.Adam(encoder_batch.parameters(), lr=learning_rate)
dec_optim = optim.Adam(decoder_batch.parameters(), lr=learning_rate)
criterion = nn.NLLLoss(ignore_index=PAD_token)

plot_losses = []
start = time.time()

print(f"🚀 Запуск MASKED обучения...")

for iter in range(1, n_iters + 1):
    indices = torch.randint(0, len(train_input_gpu), (BATCH_SIZE,), device=device)
    input_batch = train_input_gpu[indices].transpose(0, 1)
    target_batch = train_target_gpu[indices].transpose(0, 1)
    
    # Используем новую функцию train_batch_packed
    loss = train_batch_packed(input_batch, target_batch, encoder_batch, decoder_batch,
                       enc_optim, dec_optim, criterion)
    
    plot_losses.append(loss)

    if iter % 1000 == 0:
        speed = (iter * BATCH_SIZE) / (time.time() - start)
        print(f'{iter} | Loss: {loss:.5f} | Speed: {speed:.0f} samples/sec')

print("✅ Обучение завершено.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Функция сглаживания (свертка)
def moving_average(data, window_size=50):
    # window_size=50 означает, что каждая точка графика - это среднее за 50 шагов
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

# Применяем фильтр к твоему списку ошибок
smooth_losses = moving_average(plot_losses)

# Рисуем
plt.figure(figsize=(10, 5))
plt.plot(smooth_losses, label='Smoothed Loss')
plt.title("График обучения (Скользящее среднее)")
plt.xlabel("Итерации")
plt.ylabel("Ошибка")
plt.grid(True, alpha=0.3) # Добавим сетку для наглядности
plt.legend()
plt.show()

In [ ]:
import matplotlib.ticker as ticker


# === ОБНОВЛЕННАЯ ВАЛИДАЦИЯ (С поддержкой масок) ===

def evaluate(input_sentence):
    encoder_batch.eval()
    decoder_batch.eval()
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_sentence)
        # Длина всегда равна реальной длине тензора
        length = torch.tensor([input_tensor.size(0)]) 
        
        encoder_hidden = encoder_batch.initHidden(batch_size=1)
        
        # Передаем length!
        encoder_outputs, encoder_hidden = encoder_batch(input_tensor, length, encoder_hidden)
        
        # ... (остальной код без изменений) ...
        # (Не забудь скопировать остаток функции из прошлой версии, где мы делали маски)
        # Для краткости: используй ту же логику proj_encoder_outputs и encoder_mask, что была
        
        # --- (Копипаст старого кода ниже для полноты) ---
        proj_encoder_outputs = torch.zeros(decoder_batch.max_length, 1, encoder_batch.hidden_size, device=device)
        out_len = encoder_outputs.size(0)
        limit_copy = min(out_len, decoder_batch.max_length)
        proj_encoder_outputs[:limit_copy, :, :] = encoder_outputs[:limit_copy, :, :]

        encoder_mask = torch.zeros(1, decoder_batch.max_length, dtype=torch.bool, device=device)
        encoder_mask[:, :limit_copy] = True

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []
        decoder_attentions = torch.zeros(decoder_batch.max_length, decoder_batch.max_length)

        for di in range(decoder_batch.max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder_batch(
                decoder_input, decoder_hidden, proj_encoder_outputs, encoder_mask=encoder_mask)
            decoder_attentions[di] = decoder_attention.data[0]
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(idx2char[topi.item()])
            decoder_input = topi.detach().transpose(0, 1)
            
    encoder_batch.train()
    decoder_batch.train()
    return "".join(decoded_words), decoder_attentions[:di + 1]

# Эти функции остаются без изменений, но зависят от новой evaluate
def evaluate_randomly(n=5):
    print(f"\n🎓 ЭКЗАМЕН (Masked Model):")
    correct = 0
    for i in range(n):
        pair = random.choice(test_data)
        src = pair[0]
        tgt_real = pair[1]
        
        output_words, _ = evaluate(src)
        
        tgt_clean = tgt_real.replace('<','').replace('>','')
        out_clean = output_words.replace('<EOS>','')
        
        is_match = "✅" if tgt_clean == out_clean else f"❌ ({tgt_clean})"
        if tgt_clean == out_clean: correct += 1
        print(f"{src.ljust(15)} -> {out_clean.ljust(12)} {is_match}")
    print(f"Точность: {correct}/{n}")

def showAttention(input_sentence):
    output_words, attentions = evaluate(input_sentence)
    clean_output = output_words.replace('<EOS>', '')
    
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    ax.set_xticklabels([''] + list(input_sentence), rotation=90)
    ax.set_yticklabels([''] + list(clean_output))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

# === ФИНАЛЬНЫЙ ТЕСТ ===
evaluate_randomly(10)
print("\n--- Проверка Внимания ---")
showAttention("1 Jan 2024")